In [1]:
from Vissim_env_class import environment
from MasterDQN_Agent import MasterDQN_Agent
from collections import deque

# Network Specific Libraries
from Balance_Functions import balance_dictionary

# General Libraries
import numpy as np 
import pylab as plt
import os
import csv
import pandas as pd
import json
import pickle

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [17]:
model_name  = 'Single_Cross_Triple'
#vissim_working_directory =  'C:\\Users\\Rzhang\\Desktop\\MLforFlowOptimisationOrigine\\Vissim\\'
vissim_working_directory = "C:\\Users\\acabrejasegea\\Desktop\\15_Timescales_utc\\Timescales_and_Discount_Factor_in_UTC"

sim_length = 3601
timesteps_per_second = 1
learning_iterations = 4
actions_set = "all_actions"

agent_type = "DuelingDDQN"
Session_ID = "SCT_8ac_MemGen_Q_Range_10_gamma77"
#ession_ID = "Single_Cross_Triple8_actions_DuelingDDQN20c10"

## DQN Hyperaramenters
copy_weights_frequency = 20 # On a successfull run I copied the weight every 50
exam_frequency = 40
final_exams = 10

PER_activated = False
memory_size = 512
batch_size = 128

gamma = 0.77
alpha = 0.001

# Source Memory partitioning
group_size = 512
overlap = 128

Random_Seed = 777

epsilon_sequence = np.zeros(5000).tolist()

# all controller actions
Single_Cross_Triple_dictionary8 =\
{\
    # Controller Number 0 
   'junctions' : {0 : {'default_actions' :  {    0 : [1,1,1,0,0,0,0,0,0,0,0,0],
                                             1 : [0,0,0,1,1,1,0,0,0,0,0,0],
                                             2 : [0,0,0,0,0,0,1,1,1,0,0,0],
                                             3 : [0,0,0,0,0,0,0,0,0,1,1,1],
                                             4 : [1,0,0,0,0,0,1,0,0,0,0,0],
                                             5 : [0,0,0,1,0,0,0,0,0,1,0,0],
                                             6 : [0,1,1,0,0,0,0,1,1,0,0,0],
                                             7 : [0,0,0,0,1,1,0,0,0,0,1,1]},
         
         'all_actions' :        {            0 : [1,1,1,0,0,0,0,0,0,0,0,0],
                                             1 : [0,0,0,1,1,1,0,0,0,0,0,0],
                                             2 : [0,0,0,0,0,0,1,1,1,0,0,0],
                                             3 : [0,0,0,0,0,0,0,0,0,1,1,1],
                                             4 : [1,0,0,0,0,0,1,0,0,0,0,0],
                                             5 : [0,0,0,1,0,0,0,0,0,1,0,0],
                                             6 : [0,1,1,0,0,0,0,1,1,0,0,0],
                                             7 : [0,0,0,0,1,1,0,0,0,0,1,1]},
 
         'link' : [1, 3, 5, 7],
         'lane' : ['1-1', '1-2', '1-3', '3-1', '3-2', '3-3', '5-1', '5-2', '5-3', '7-1', '7-2', '7-3'],
         
         'controled_by_com' : True,
         'agent_type' : agent_type,
         'green_time' : 6,
         'redamber_time' : 0,
         'amber_time' : 3, 
         'red_time' : 0,
         'state_size' : [13],
         'state_type' : 'QueuesSig',
         'reward_type' : 'Queues' ,
         'queues_counter_ID' : [1,2,3,4,5,6,7,8,9,10,11,12]             
         }
    },
   'demand' : { 'default' : [400, 400, 400, 400],
             
             0 : [300,300,300,300],
             1 : [600,600,600,600],
             2 : [1350,750,1350,750],
             3 : [1500,750,1500,750],
             4 : [1050,750,1050,750],
             5 : [750,1050,750,1050],
             6 : [750,1500,750,1500],
             7 : [750,1350,750,1350],
             8 : [600,600,600,600],
             9 : [300,300,300,300]
            }
}

In [18]:
def plot_results_train():
    ## AGENT TRAINING RESULTS
    # Path to results folder
    results_path = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID)
    
    # Loop over each agent
    for idx , agent in Single_Cross_Triple8_MultiDQN_Agents.Agents.items():
        intersection_number_in_vissim = Single_Cross_Triple8_MultiDQN_Agents.Agents[0].signal_id + 1
        print("Intersection "+str(intersection_number_in_vissim))
        
        ## SAVE TRAINING DATA TO JSON.
        json_filename = "Agent{}_Loss_average_reward.json".format(intersection_number_in_vissim)
        Loss_reward = dict()   
        # Loss dictionary
        for epoch, loss in enumerate(agent.loss):
            loss_dict = { epoch : loss }
        Loss_reward['Agent{} loss'.format(intersection_number_in_vissim)] = loss_dict
        # Reward dictionary            
        episode = [i for i in range(len(agent.reward_storage))]
        Loss_reward['Agent{} Average_Reward'.format(intersection_number_in_vissim)] = agent.reward_storage
        # Store as JSON
        with open(results_path + "/" + json_filename, 'w') as f:
            json.dump(Loss_reward, f)
        print("Agent {}: Training Loss and Average Reward during training successfuly saved to file:".format(intersection_number_in_vissim))
        print(results_path + "/" + json_filename)
        
        ## LOADING DATA FROM JSON
        #results_path = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID)
        #dictionary_filename = "Agent{}_Loss_average_reward.json".format(intersection_number_in_vissim)
        #with open(results_path + "/" + json_filename, 'r') as fp:
        #    data = json.load(fp)
        #print(data)
        
        ## TRAINING PLOTS
        loss_plot_filename  = "Agent{}_Loss.png".format(intersection_number_in_vissim)
        reward_plot_filename  = "Agent{}_average_reward.png".format(intersection_number_in_vissim) 
        
        ## Loss Plot
        plt.figure('LossAgent'+str(idx),figsize=(16,9))
        plt.plot(agent.loss)
        #plt.yscale('log')
    
        plt.xlabel('Training Epoch',fontsize=18)
        plt.ylabel('Loss',fontsize=18)
        plt.title('Agent {} Loss over training'.format(intersection_number_in_vissim),fontsize=18)
        plt.savefig(results_path + "/" + loss_plot_filename)
    
        ## Average Reward Plot
        plt.figure('RewardAgent'+str(idx),figsize=(16,9))
        plt.plot(agent.reward_storage)
        plt.xlabel('Training Episode',fontsize=18)
        plt.ylabel('Average reward',fontsize=18)
        plt.title('Agent {} average reward over training'.format(intersection_number_in_vissim),fontsize=18)
        plt.savefig(results_path + "/" + reward_plot_filename)

def plot_results_test(ID):
    time = [t for t in range(len(Single_Cross_Triple8_MultiDQN_Agents.Cumulative_Totale_network_stop_delay))]

    # Queues ovzer time for each junction
    for idx, queues in Single_Cross_Triple8_MultiDQN_Agents.Episode_Queues.items():
        queues = np.array(queues)
        queues = queues.T
        
        plt.figure('1'+str(idx),figsize=(16, 9))
        
        Queuesdf = pd.DataFrame()
        
        Queuesdf['Time'] = time
        Queues_legend = []
        
        for i, queue in enumerate(queues):
            plt.plot(queue, label = "Queue"+str(i))
            Queuesdf[str(i)] = queue
            Queues_legend.append("Queue"+str(i))
            
        plt.xlabel('Time [s]',fontsize=18)
        plt.ylabel('Queue Length',fontsize=18)
        plt.title('Junction {} Queue length'.format(idx),fontsize=18)
        #plt.gca().legend(Queues_legend)
        
        Path = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID,\
                            "Junction{}_Queues_{}.png".format(idx,ID))               
        plt.savefig(Path)
        
        csv_Path = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID,\
                            "Junction{}_Queues_{}.csv".format(idx,ID))        
       
        Queuesdf.to_csv(csv_Path,index=False)
        plt.legend()
            
            
    # Accumulated delay over time for each junction
    for idx, delay in Single_Cross_Triple8_MultiDQN_Agents.Cumulative_Episode_Delays.items():
        
        plt.figure('2'+str(idx),figsize=(16, 9))
        
        delaydf = pd.DataFrame()   
        delaydf['Time'] = time
        delaydf['Junction {} delay'.format(idx)] = delay
        
        plt.plot(delay)
        plt.xlabel('Time [s]',fontsize=18)
        plt.ylabel('Accumulated Delay',fontsize=18)
        plt.title('Junction {} Delay'.format(idx),fontsize=18)
        plt.gca().legend('Junction accumulated delay')
        
        Path = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID,\
                                "Junction{}_Cumulative_Delay_{}.png".format(idx,ID))
        csv_Path = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID,\
                            "Junction{}_Cumulative_Delay_{}.csv".format(idx,ID))   
        plt.savefig(Path)
        delaydf.to_csv(csv_Path,index=False)
        
        
    # Accumulated stop delay over time for each junction
    for idx, stop_delay in Single_Cross_Triple8_MultiDQN_Agents.Cumulative_Episode_stop_Delays.items():
        
        plt.figure('3'+str(idx),figsize=(16, 9))
        
        stop_delaydf = pd.DataFrame()   
        stop_delaydf['Time'] = time
        stop_delaydf['Junction {} stop delay'.format(idx)] = stop_delay
    
        plt.plot(stop_delay)
        plt.xlabel('Time [s]',fontsize=18)
        plt.ylabel('Accumulated Stop Delay',fontsize=18)
        plt.title('Junction {} Stop Delay'.format(idx),fontsize=18)
        plt.gca().legend('Junction accumulated Stop delay')
        
        Path = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID,\
                                "Junction{}_Cumulative_stop_Delay_{}.png".format(idx,ID))
        csv_Path = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID,\
                            "Junction{}_Cumulative_stop_Delay_{}.csv".format(idx,ID))
        plt.savefig(Path)
        stop_delaydf.to_csv(csv_Path,index=False)

In [82]:
Single_Cross_Triple8_MultiDQN_Agents = MasterDQN_Agent(model_name, vissim_working_directory,\
                                                       sim_length, Single_Cross_Triple_dictionary8,\
                                                       actions_set, gamma, alpha, agent_type,\
                                                       memory_size, PER_activated, batch_size,\
                                                       learning_iterations, copy_weights_frequency,\
                                                       epsilon_sequence,Random_Seed,\
                                                       timesteps_per_second, Session_ID,\
                                                       verbose = True)

INTERSECTION 0: SETTING UP AGENT
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 13)]         0                                            
__________________________________________________________________________________________________
dense_72 (Dense)                (None, 48)           672         input_13[0][0]                   
__________________________________________________________________________________________________
dense_73 (Dense)                (None, 48)           2352        dense_72[0][0]                   
__________________________________________________________________________________________________
dense_76 (Dense)                (None, 48)           2352        dense_73[0][0]                   
__________________________________________________________

In [20]:
# Load external memory
#
#############################################
# external_memory[i]    = i-th memory       #
# external_memory[i][0] = i-th state        #
# external_memory[i][1] = i-th action       #
# external_memory[i][2] = i-th reward       #
# external_memory[i][3] = i-th next_state   # 
# external_memory[i][4] = done?             #
#############################################

filename1 = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID) + "\\memoryD3QN_400.pkl"
print("Attempting to load file:")
print(filename1 + "\n")
initial_memory = list(pickle.load(open(filename1, 'rb')))
print("400 Memory Loaded Successfully")
partitioned_initial_memory= [initial_memory[i:i+group_size] for i in range(0, len(initial_memory), group_size-overlap)]
print("400 Memory is comprised of {} groups \n".format(len(partitioned_initial_memory)))
low_demand_limit = len(partitioned_initial_memory)

#############################################
filename = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID) + "\\memoryD3QN_600.pkl"
print("Attempting to load file:")
print(filename + "\n")
memory2 = list(pickle.load(open(filename, 'rb')))
print("600 Long Memory Loaded Successfully")
partitioned_memory2= [memory2[i:i+group_size] for i in range(0, len(memory2), group_size-overlap)]
print("600 Long Memory Partitioned Successfully into {} groups \n".format(len(partitioned_memory2)))
mid_demand_limit =  len(partitioned_memory2)

#############################################
filename2 = os.path.join(vissim_working_directory, model_name, "Agents_Results", agent_type, Session_ID) + "\\memoryD3QN_800.pkl"
print("Attempting to load file:")
print(filename2 + "\n")
memory = list(pickle.load(open(filename2, 'rb')))
print("800 Memory Loaded Successfully")
partitioned_memory= [memory[i:i+group_size] for i in range(0, len(memory), group_size-overlap)]
print("800 Memory is comprised of {} groups \n".format(len(partitioned_memory)))
total_episodes = len(partitioned_memory)

Attempting to load file:
C:\Users\acabrejasegea\Desktop\15_Timescales_utc\Timescales_and_Discount_Factor_in_UTC\Single_Cross_Triple\Agents_Results\DuelingDDQN\SCT_8ac_MemGen_Q_Range_10_gamma77\memoryD3QN_400.pkl

400 Memory Loaded Successfully
400 Memory is comprised of 1092 groups 

Attempting to load file:
C:\Users\acabrejasegea\Desktop\15_Timescales_utc\Timescales_and_Discount_Factor_in_UTC\Single_Cross_Triple\Agents_Results\DuelingDDQN\SCT_8ac_MemGen_Q_Range_10_gamma77\memoryD3QN_600.pkl

600 Long Memory Loaded Successfully
600 Long Memory Partitioned Successfully into 2236 groups 

Attempting to load file:
C:\Users\acabrejasegea\Desktop\15_Timescales_utc\Timescales_and_Discount_Factor_in_UTC\Single_Cross_Triple\Agents_Results\DuelingDDQN\SCT_8ac_MemGen_Q_Range_10_gamma77\memoryD3QN_800.pkl

800 Memory Loaded Successfully
800 Memory is comprised of 3406 groups 



In [81]:
# Training Loop
exams = 0
volume = 400

# Start Environment
Single_Cross_Triple8_MultiDQN_Agents.env = environment(model_name, vissim_working_directory, sim_length,\
                                                       Single_Cross_Triple_dictionary8, actions_set, Random_Seed,\
                                                       timesteps_per_second = timesteps_per_second, mode = 'training',\
                                                       delete_results = True, verbose = True)
# Training Loop
for episode in range(total_episodes):
    # Train and evaluate accordingly to demand levels that generated memories
    if episode > low_demand_limit:
        volume = 600
    elif episode > mid_demand_limit:
        volume = 800
    Single_Cross_Triple8_MultiDQN_Agents.env.change_demand(volume)
    
    # Set up Memory
    print("Round {}/{}: Adding {} memories from Deque into Agent".format(episode, len(partitioned_memory)-1,\
                                                                                       len(partitioned_memory[episode])))
    Single_Cross_Triple8_MultiDQN_Agents.Agents[0].memory = deque(maxlen=memory_size)
    for idx,sarsd in enumerate(partitioned_memory[episode]):
        s = sarsd[0]
        a = sarsd[1]
        r = sarsd[2]
        ns = sarsd[3]
        d = sarsd[4]
        Single_Cross_Triple8_MultiDQN_Agents.Agents[0].remember_external(s,a,r,ns,d)
    
    # Stochastic Gradient Descent on weights
    print("Memories succesfully loaded. Performing SGD on weights...")
    for iteration in range(learning_iterations):
        Single_Cross_Triple8_MultiDQN_Agents.Agents[0].learn_batch(128, 1)
    
    # Every 20 iterations, copy weights
    if episode%copy_weights_frequency == 0:
        Single_Cross_Triple8_MultiDQN_Agents.Agents[0].copy_weights()
    # Every so often, check agent performance
    if episode%exam_frequency == 0:
        print("Starting exam run...")
        exams += 1
        Single_Cross_Triple8_MultiDQN_Agents.external_training()

# After all memories are processed, check agent performance one last time, close environment, plot training and save.
Single_Cross_Triple8_MultiDQN_Agents.external_training()
Single_Cross_Triple8_MultiDQN_Agents.env = None
plot_results_train()
Single_Cross_Triple8_MultiDQN_Agents.save(len(partitioned_memory))

Working Directory set to: C:\Users\acabrejasegea\Desktop\15_Timescales_utc\Timescales_and_Discount_Factor_in_UTC
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Single_Cross_Triple.inpx ...
Model File load process successful.
Simulation length set to 3601 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                COM SETUP COMPLETE                   *
*                                                     *
*******************************************************

Random seed set in simulator. Random Seed = 777
Deploying Network Parser...
Successful Network Crawl: Identified SignalControllers, Links, Lanes and Vehicle Inputs.

Setting Simulation mode to: training
Starting Deployments of Signal C

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x00000255B7F16040>
Traceback (most recent call last):
  File "C:\Users\acabrejasegea\AppData\Local\Continuum\anaconda3\envs\vissim\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 537, in __del__
    gen_dataset_ops.delete_iterator(
  File "C:\Users\acabrejasegea\AppData\Local\Continuum\anaconda3\envs\vissim\lib\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1137, in delete_iterator
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt: 


1/1 - 0s - loss: 0.1749
1/1 - 0s - loss: 0.1978
Round 13/3917: Adding 512 memories from Deque into Agent
Memories succesfully loaded. Performing SGD on weights...
1/1 - 0s - loss: 0.2990
1/1 - 0s - loss: 0.2781
1/1 - 0s - loss: 0.2440
1/1 - 0s - loss: 0.1874
Round 14/3917: Adding 512 memories from Deque into Agent
Memories succesfully loaded. Performing SGD on weights...
1/1 - 0s - loss: 0.2032
1/1 - 0s - loss: 0.2089
1/1 - 0s - loss: 0.1848


KeyboardInterrupt: 

In [84]:
for idx,sarsd in enumerate(partitioned_memory[100]):
        s = sarsd[0]
        a = sarsd[1]
        r = sarsd[2]
        ns = sarsd[3]
        d = sarsd[4]
        print(r)
        Single_Cross_Triple8_MultiDQN_Agents.Agents[0].remember_external(s,a,r,ns,d)

-0.14780602968150833
-0.14777865582803798
-0.164339755248815
-0.15215259699245665
-0.1590434499333373
-0.12433053250840441
-0.12222333157581236
-0.11393233670956557
-0.11000776448406088
-0.12469436594966231
-0.1017449009780047
-0.11601364943824716
-0.1342060568971819
-0.11602173866742627
-0.09281686681646723
-0.09487013236101444
-0.06593489742045096
-0.045499616583588
-0.04412616988534684
-0.04402988792714658
-0.06111260769744874
-0.08366964823997465
-0.08589941639910689
-0.10557359903785406
-0.07844110023027986
-0.05651253661243175
-0.042310270401599605
-0.03651513583466542
-0.04735808539384769
-0.04618237201620685
-0.04556840214778619
-0.05246261775526561
-0.06110871886601076
-0.03935936802166282
-0.039215737125532855
-0.04878604798980922
-0.046471079129165656
-0.038390564297172615
-0.0457598310074511
-0.04414878432088778
-0.04414878432088778
-0.057901282329953875
-0.05995268087969951
-0.05001737493918991
-0.051961159980902755
-0.05422852468238133
-0.05114301265187289
-0.054916647740

In [42]:
Single_Cross_Triple8_MultiDQN_Agents.Agents[0].model.predict(partitioned_memory[1][0][0])

array([[ 3.5444324 ,  0.37529433,  2.6801221 ,  1.2990544 , -0.3484372 ,
        -1.3430383 ,  2.6089725 ,  0.6216017 ]], dtype=float32)

In [52]:
Single_Cross_Triple8_MultiDQN_Agents.Agents[0].memory

deque([[array([[ 5.1669398 ,  6.03904212,  5.39484066,  0.        , 66.35606386,
                69.8577421 , 16.53131033, 15.2285188 , 57.82891855,  0.        ,
                 0.        ,  5.12557281,  2.        ]]),
        2,
        -0.12778132570146147,
        array([[ 0.        ,  6.03904212,  5.39484066,  0.        , 66.35606386,
                75.38905133,  0.        ,  0.        , 42.99549638,  0.        ,
                18.4250277 , 15.40686422,  4.        ]]),
        False],
       [array([[ 0.        ,  6.03904212,  5.39484066,  0.        , 66.35606386,
                75.38905133,  0.        ,  0.        , 42.99549638,  0.        ,
                18.4250277 , 15.40686422,  4.        ]]),
        4,
        -0.12772767975072793,
        array([[ 0.        ,  0.        ,  0.        ,  0.        , 66.35606386,
                75.38905133,  0.        ,  0.        , 45.64142573,  0.        ,
                18.29005236, 24.23323027,  6.        ]]),
        False],
      

In [79]:
for i in range(100):
    Single_Cross_Triple8_MultiDQN_Agents.Agents[0].learn_batch(128,1)

1/1 - 0s - loss: 6.7214
1/1 - 0s - loss: 7.6331
1/1 - 0s - loss: 10.2334
1/1 - 0s - loss: 6.7004
1/1 - 0s - loss: 9.4591
1/1 - 0s - loss: 9.3218
1/1 - 0s - loss: 6.4141
1/1 - 0s - loss: 8.3376
1/1 - 0s - loss: 9.0181
1/1 - 0s - loss: 7.7440
1/1 - 0s - loss: 6.3218
1/1 - 0s - loss: 5.8420
1/1 - 0s - loss: 5.1538
1/1 - 0s - loss: 7.6061
1/1 - 0s - loss: 8.3363
1/1 - 0s - loss: 5.3753
1/1 - 0s - loss: 7.7862
1/1 - 0s - loss: 9.2222
1/1 - 0s - loss: 4.0302
1/1 - 0s - loss: 4.5823
1/1 - 0s - loss: 4.0582
1/1 - 0s - loss: 3.7086
1/1 - 0s - loss: 3.8591
1/1 - 0s - loss: 3.5131
1/1 - 0s - loss: 6.2999
1/1 - 0s - loss: 3.8790
1/1 - 0s - loss: 2.9236
1/1 - 0s - loss: 3.7876
1/1 - 0s - loss: 3.5163
1/1 - 0s - loss: 2.7101
1/1 - 0s - loss: 3.4376
1/1 - 0s - loss: 4.0634
1/1 - 0s - loss: 3.7346
1/1 - 0s - loss: 3.9315
1/1 - 0s - loss: 2.4109
1/1 - 0s - loss: 3.9109
1/1 - 0s - loss: 3.8522
1/1 - 0s - loss: 2.7774
1/1 - 0s - loss: 3.0254
1/1 - 0s - loss: 3.5140
1/1 - 0s - loss: 3.9187
1/1 - 0s - loss

In [ ]:
Single_Cross_Triple8_MultiDQN_Agents.load(len(partitioned_memory), best=False)

In [ ]:
Single_Cross_Triple8_MultiDQN_Agents.demo()

In [ ]:
# Perform Evaluation
print("\n" + "Starting final exams...")
for repetition in range(3*final_exams):
    if repetition < final_exams:
        volume = 400
    elif repetition >= final_exams and repetition < (2*final_exams):
        volume = 800
    elif repetition >= (2*final_exams):
        volume = 1200
        
    Single_Cross_Triple8_MultiDQN_Agents.test_demand(volume)
    plot_results_test(str(volume)+"_"+str(repetition))


In [ ]:
from csv import reader

demand = 400

storage = {i:[] for i in range(10)}

for i in range(10):
    folder05 = "C:\\Users\\acabrejasegea\Desktop\\15_Timescales_utc\\Timescales_and_Discount_Factor_in_UTC\\Single_Cross_Triple\\Agents_Results\\DuelingDDQN\\SCT_8ac_MemGen_Q_2_Res_Gamma05\\"
    folder85 = "C:\\Users\\acabrejasegea\Desktop\\15_Timescales_utc\\Timescales_and_Discount_Factor_in_UTC\\Single_Cross_Triple\\Agents_Results\\DuelingDDQN\\SCT_8ac_MemGen_Q_2_Res_Gamma85\\"
    folder95 = "C:\\Users\\acabrejasegea\Desktop\\15_Timescales_utc\\Timescales_and_Discount_Factor_in_UTC\\Single_Cross_Triple\\Agents_Results\\DuelingDDQN\\SCT_8ac_MemGen_Q_2_Res_Gamma95\\"

    file05 = "Junction0_Cumulative_Delay_"
    # read csv file as a list of lists
    with open(folder05+file05+str(demand)+"_{}.csv".format(i), 'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Pass reader object to list() to get a list of lists
        storage[i] = list(csv_reader)



In [ ]:
delay = [storage[0][idx][1] for idx in range(1,len(storage[0]))]
time = [storage[0][idx][0] for idx in range(1,len(storage[0]))]

In [ ]:
delay_r = [np.round(item,2) for item in delay]

In [ ]:
plt.plot(data_plot[1:-1])

In [ ]:
import matplotlib as mtp
